In [108]:
import pandas as pd
import unicodedata
from nltk.corpus import stopwords
import nltk
import re
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer 

nltk.download('wordnet')
nltk.download("stopwords")
lemmatizer = WordNetLemmatizer()
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kadss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kadss\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:

#read data of city

boston = pd.read_csv('Data/2CVTweets/Boston2C.csv', sep=";",header=None,error_bad_lines=False)
boston = boston.drop(boston.columns[0], axis=1) ## index column
boston.columns=['tweet','class']

b'Skipping line 3: expected 3 fields, saw 4\nSkipping line 5: expected 3 fields, saw 4\nSkipping line 8: expected 3 fields, saw 4\nSkipping line 9: expected 3 fields, saw 4\nSkipping line 12: expected 3 fields, saw 4\nSkipping line 19: expected 3 fields, saw 4\nSkipping line 20: expected 3 fields, saw 4\nSkipping line 21: expected 3 fields, saw 4\nSkipping line 22: expected 3 fields, saw 4\nSkipping line 23: expected 3 fields, saw 4\nSkipping line 24: expected 3 fields, saw 4\nSkipping line 25: expected 3 fields, saw 4\nSkipping line 27: expected 3 fields, saw 4\nSkipping line 28: expected 3 fields, saw 4\nSkipping line 29: expected 3 fields, saw 4\nSkipping line 30: expected 3 fields, saw 4\nSkipping line 31: expected 3 fields, saw 4\nSkipping line 32: expected 3 fields, saw 4\nSkipping line 33: expected 3 fields, saw 4\nSkipping line 34: expected 3 fields, saw 4\nSkipping line 35: expected 3 fields, saw 4\nSkipping line 36: expected 3 fields, saw 4\nSkipping line 37: expected 3 field

In [138]:
boston.head(20)

,tweet,class,tweet_tok_lem,unigram_vec,bigram_vec,tfid,no_ques_marks,no_excl_marks,no_uppercase
0,"b'RT' b'@CommuterBoston:' b'ACCIDENT' b'(Wellesley,' b'MA):' b'RT-9' b'East' b'near' b'Cedar' b'St' b'-' b'involving' b'a' b'motorcycle' b'and' b'a' b'car' b'-' b'possible' b'lane' b'restrictions'",YES,"[b'RT, ', b, ', @, CommuterBoston, :, ', b'ACCIDENT, ', b, ', (, Wellesley, ,, ', b'MA, ), :, ', b'RT-9, ', b'East, ', b'near, ', b'Cedar, ', b'St, ', b'-, ', b'involving, ', b, ', a, ', b'motorcycle, ', b'and, ', b, ', a, ', b'car, ', b'-, ', b'possible, ', b'lane, ', b'restrictions, ']","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]]","{'b'computer': 0.0, 'b'hospital': 0.0, 'b'Trapelo': 0.0, 'b'Inadequate': 0.0, '12': 0.0, 'b'got': 0.0, ',': 0.021128899515294463, 'CSNNE': 0.0, 'b'Ave': 0.0, '7News': 0.0, 'b'overpassURL': 0.0, 'Ferry': 0.0, 'b'thief': 0.0, 'b'door': 0.0, 'b'be': 0.0, 'b'United': 0.0, 'b'Motorcyclist': 0.0, 'b'Grove': 0.0, 'b'she': 0.0, 'b'suspected': 0.0, 'b'joins': 0.0, 'b'good': 0.0, 'b'Petrobras': 0.0, 'b'more': 0.0, 'b'carjacker': 0.0, 'b'Expwy': 0.0, 'b'Waverley': 0.0, 'b'Granite': 0.0, '41': 0.0, 'b'Lt': 0.0, 'b'merchandise': 0.0, 'b'origin': 0.0, 'b'HD': 0.0, 'b'ran': 0.0, 'b'Somehow': 0.0, 'b'face': 0.0, 'b'Deadly': 0.0, 'b'white': 0.0, 'b'Wellesley': 0.0, 'MH17': 0.0, 'b'Officer': 0.0, 'b'rising': 0.0, 'b'weapon': 0.0, 'b'fence': 0.0, 'PD/FD': 0.0, '!': 0.0, 'don': 0.0, 'b'Montana': 0.0, 'b'put': 0.0, 'b'Damage': 0.0, 'b'story': 0.0, 'b'Love': 0.0, 'b'Bags': 0.0, 'b'Crippling': 0.0, 'b'fell': 0.0, 'b'Marion': 0.0, 'you': 0.0, '5:20': 0.0, 'b'sidewalk': 0.0, 'b'GM': 0.0, 'b'getting': 0.0, 'b'investig': 0.0, 'b'There': 0.0, 'b'Road': 0.0, 'b'Latest': 0.0, 'b'makes': 0.0, 'b'pictures': 0.0, 'b'Ethiopia': 0.0, 'b'Friendly': 0.0, 'b'ppl': 0.0, 'b'79': 0.0, 'b'Belmont': 0.0, 'CommuterBoston': 0.048928795567949435, 'didn': 0.0, 'b'gas': 0.0, 'b'Field': 0.0, 'Peabody': 0.0, 'b'explosion': 0.0, 'b'battling': 0.0, 'b'heard': 0.0, 'b'neighbors': 0.0, 'b'Roxbury': 0.0, 'Providence': 0.0, 'b'hot': 0.0, 'b'tow': 0.0, 'b'And': 0.0, 'matraffic': 0.0, 'b'opens': 0.0, 'b'prison': 0.0, 'b'side': 0.0, 'b'something': 0.0, 'b'Unknown': 0.0, 'b'twice': 0.0, 'b'Joe': 0.0, 'csmonitor': 0.0, 'EMT': 0.0, 'b'ordered': 0.0, 'b'READING': 0.0, 'WBTV_News': 0.0, 'b'nightmares': 0.0, ...}",0,0,0
1,b'RT' b'@CommuterBoston:' b'ACCIDENT:' b'I-290' b'East' b'before' b'I-495' b'(Exit' b'#26)' b'-' b'2-car' b'accident' b'with' b'possible' b'lane' b'blockages',YES,"[b'RT, ', b, ', @, CommuterBoston, :, ', b'ACCIDENT, :, ', b, ', I-290, ', b'East, ', b'before, ', b, ', I-495, ', b, ', (, Exit, ', b, ', #, 26, ), ', b'-, ', b, ', 2-car, ', b'accident, ', b'with, ', b'possible, ', b'lane, ', b'blockages, ']","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]]","{'b'computer': 0.0, 'b'hospital': 0.0, 'b'Trapelo': 0.0, 'b'Inadequate

## Pre-Processing

In [48]:
#unicode conversion
boston['tweet'] = (boston['tweet'].map(lambda x: unicodedata.normalize('NFKD', x))
              .str.encode('ascii', 'ignore'))

In [139]:
# slang conversion

In [49]:


from bs4 import BeautifulSoup
import requests, json
resp = requests.get('http://www.netlingo.com/acronyms.php')
soup = BeautifulSoup(resp.text, "html.parser")
slangdict= {}
key=""
value=""
for div in soup.findAll('div', attrs={'class':'list_box3'}):
    
    for li in div.findAll('li'):
        for a in li.findAll('a'):
            key =a.text
            value = li.text.split(key)[1]
            slangdict[key.lower()]=value

In [50]:
def slang_to_formal(input):
    i=1
    formal=[]
    for slang in input.split():
        i=i+1
        if slang.lower() in slangdict:
            formal.append(slangdict[slang])
        elif slang.isdigit():
            
            formal.append('D')  #not working
        else:
            formal.append(slang)
       
        
    return  " ".join(str(x) for x in formal)
            
        

In [51]:
boston['tweet'] = boston.apply(lambda row: slang_to_formal(row['tweet']), axis=1)

In [52]:
#replacing URL and digits

In [53]:
def replace_URL(tweet):
    return re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'URL', tweet)

    
   


boston['tweet'] = boston.apply(lambda row: replace_URL(row['tweet']), axis=1)

In [54]:
#remove stop words

stop = stopwords.words('english')
boston['tweet'] = boston.apply(lambda x: [item for item in x if item not in stop])

In [55]:
#tokenisation

boston['tweet_tok_lem'] = boston.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)

In [56]:
#lemmatize
def lemmatize(s):
       
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

boston['tweet_tok_lem'] = boston.apply(lambda row: lemmatize(row['tweet_tok_lem']), axis=1)


## Feature Extraction

In [87]:
# unigram and bigram features

# freq vector unigram

def initialize_ngram(corpus):
    v1 = CountVectorizer()
    v2 = CountVectorizer(2)
    
    v1.fit(corpus)
    v2.fit(corpus)
    return v1,v2

def convert_ngram_vec(text,v):
    text=' '.join(text)
    s=[]
    s.append(text)
    return v.transform(s).toarray()
    
all_tweets_corpus=boston['tweet'].tolist()

v1,v2=initialize_ngram(all_tweets_corpus)

boston['unigram_vec'] = boston.apply(lambda row: convert_ngram_vec(row['tweet_tok_lem'],v1), axis=1)

boston['bigram_vec'] = boston.apply(lambda row: convert_ngram_vec(row['tweet_tok_lem'],v2), axis=1)


In [140]:
#tfidf scores

In [91]:

uniqueWords=set()
for tweet in boston['tweet_tok_lem']:
    uniqueWords = uniqueWords.union(set(tweet))

In [103]:
def compute_num_tokens(tweet_token_list):
    num_tokens = dict.fromkeys(uniqueWords, 0)
    for token in tweet_token_list:
        num_tokens[token] += 1
    return num_tokens

def apply_tfidf(tweet_token_list,idfs):
    num_tokens=compute_num_tokens(tweet_token_list)
    tfDict=computeTF(num_tokens,tweet_token_list)  
    
    return computeTFIDF(tfDict,idfs)

In [104]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [105]:
def computeIDF(documents):

    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [106]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [109]:
idfs = computeIDF([compute_num_tokens(tweet) for tweet in boston['tweet_tok_lem']])

In [111]:
boston['tfid'] = boston.apply(lambda row: apply_tfidf(row['tweet_tok_lem'],idfs), axis=1)

In [135]:
#syntactic features

In [137]:
boston['no_ques_marks']=boston.apply(lambda row: (row['tweet'].count("?")), axis=1)
boston['no_excl_marks']=boston.apply(lambda row: (row['tweet'].count("!")), axis=1)

boston['no_uppercase']=boston.apply(lambda row: sum(map(str.isupper, row['tweet'].split())), axis=1)